# Import modules

In [ ]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sb
import shap
from Robert_functions import *

# Code

In [ ]:
# set some information about file names for the databases and random seeds used for the RF model

w_dir = os.getcwd()

# name of the csv containing the database generated previously by Robert, without the CSV extension. 
# For example: csv_name = 'Phenolic_data_final_dataset' 
# csv_training = 'CSV_NAME_final_dataset'
csv_training = 'Robert_example_final_dataset'

# if there is an external set available, specify
# csv_test = 'CSV_TEST_NAME'
csv_test = 'Robert_example_test'

# csv file containing the optimal parameters for the ML model generated previously by Robert
csv_pred_params = 'Predictor_parameters'

# specify the response value (y), for example: response_value = 'activation_barrier_kcal/mol'
# response_value = 'YOUR_Y_VALUE'
response_value = 'Target_values'

# specify columns of the csv to drop from the descriptors but to keep in the final database
# (i.e. reaction names). For example: fixed_descriptors = ['Name','SMILES','YSI/MW','YSI','CN','MW','weakest_bondtype']
# fixed_descriptors = ['DESC1','DESC2','ETC']
fixed_descriptors = ['Name']


In [ ]:
# loads the RF parameters and dataset

# load the RF parameters

# name of the file containing the parameters
parameters_df = pd.read_csv(csv_pred_params+'.csv')
    
# Retrieve the optimal number for the different parameters
n_estimators = int(parameters_df['n_estimators'][0])
max_depth = int(parameters_df['max_depth'][0])
max_features = int(parameters_df['max_features'][0])
train_proportion = float(parameters_df['train_proportion'][0].split('%')[0])
model_type = parameters_df['model_type'][0]
prediction_type = parameters_df['prediction_type'][0]
random_init = parameters_df['random_init'][0]
train_proportion = int(parameters_df['train_proportion'][0].split('%')[0])

print('\nThe best parameters for the random forest model are:',
     '\nNumber of estimators:',str(n_estimators),
     '\nMax. depth:',str(max_depth),
     '\nMax. features:',str(max_features)) 

# load the dataset

df_model = pd.read_csv(csv_training+'.csv')

training_data = df_model[df_model.Set == 'Training']
validation_data = df_model[df_model.Set == 'Validation']

X_train = training_data.drop(fixed_descriptors+[response_value]+['Set']+[f'Predicted {response_value}'], axis=1)
X_validation = validation_data.drop(fixed_descriptors+[response_value]+['Set']+[f'Predicted {response_value}'], axis=1)

y_train = training_data[response_value]
y_validation = validation_data[response_value]

print('\nAmount of loaded datapoints for the model:',
     '\nTraining set:',len(training_data),'points (', round((len(training_data)/len(df_model))*100,2),'%)',
     '\nValidation set:',len(validation_data),'points (', round((len(validation_data)/len(df_model))*100,2),'%)') 


In [ ]:
# SHAP analysis of the most important features and the impact they have on the predicted values

# standardizes the data sets using the mean and standard dev from the train set
Xmean = X_train.mean(axis=0)
Xstd = X_train.std(axis=0)
X_train_scaled = (X_train - Xmean) / Xstd

if max_features <= len(X_train.columns):
    RF = RandomForestRegressor(random_state=random_init,
        n_estimators=n_estimators, max_features=max_features,
        max_depth=max_depth)
else:   
    RF = RandomForestRegressor(random_state=random_init,
        n_estimators=n_estimators, max_features=len(X_train.columns),
        max_depth=max_depth)

# Fit the RF model with the training set
RF.fit(X_train_scaled, y_train)  

explainer = shap.TreeExplainer(RF)
shap_values = explainer.shap_values(X_train_scaled,approximate=True)
        
fig = shap.summary_plot(shap_values, X_train_scaled,max_display=10,show=False)

# save the image as a file
plt.savefig('RF SHAP importances', dpi=400, bbox_inches='tight')

In [ ]:
# add an external test set

# load the dataset
test_set = pd.read_csv(csv_test+'.csv')

X_test, y_test = pd.DataFrame(), pd.DataFrame()
# only keep the columns used to train the RF model
for column in df_model.columns:
    if column not in fixed_descriptors+[response_value,'Set',f'Predicted {response_value}']:
        X_test[column] = test_set[column]

X_test_scaled = (X_test - Xmean) / Xstd

y_test = test_set[response_value]


In [ ]:
# run the model with external test set

if prediction_type == 'reg':
    # calculate R2, MAE and RMSE for train and test sets
    r2_train,mae_train,rmse_train,r2_test,mae_test,rmse_test,y_pred_train,y_pred_test = predictor_workflow(random_init,model_type,parameters_df,X_train_scaled,y_train,X_test_scaled,y_test,prediction_type,train_proportion)
    # print stats
    print_model_stats(model_type,X_train_scaled,X_test_scaled,r2_train,mae_train,rmse_train,r2_test,mae_test,rmse_test,prediction_type,None,None,'Robert_results_test_set.txt')

elif prediction_type == 'clas':
    # calculate accuracy, F1 score and MCC for train and test sets
    accuracy_train,f1score_train,mcc_train,accuracy_test,f1score_test,mcc_test,y_pred_train,y_pred_test = predictor_workflow(random_init,model_type,parameters_df,X_train_scaled,y_train,X_test_scaled,y_test,prediction_type,train_proportion)
    # print stats
    print_model_stats(model_type,X_train_scaled,X_test_scaled,accuracy_train,f1score_train,mcc_train,accuracy_test,f1score_test,mcc_test,prediction_type,None,None,'Robert_results_test_set.txt')


In [ ]:
# Plot training, validation and test sets

y_train = training_data[response_value]
y_validation = validation_data[response_value]

y_pred_train = training_data[f'Predicted {response_value}']
y_pred_validation = validation_data[f'Predicted {response_value}']

if prediction_type == 'reg':
    sb.set(font_scale=1.2, style="ticks") #set styling preferences

    Plotdata_train = {'y_train': y_train, 'y_pred_train': y_pred_train} 
    Plotdata_validation = {'y_validation': y_validation, 'y_pred_validation': y_pred_validation}
    Plotdata_test = {'y_test': y_test, 'y_pred_test': y_pred_test}

    df_train = pd.DataFrame.from_dict(Plotdata_train)
    df_validation = pd.DataFrame.from_dict(Plotdata_validation)
    df_test = pd.DataFrame.from_dict(Plotdata_test)

    # Build the plot
    # Set up some features to plot the dots
    color_train = 'b'
    color_validation = 'orange'
    color_test = 'r'
    size = 30
    alpha = 1 # from 0 (transparent) to 1 (opaque)

    # Create subplot with a certain size and title
    fig, ax = plt.subplots(figsize=(5,5))

    # Set styling preferences
    sb.set(font_scale=1.2, style="ticks")
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    # title of the graph
    total_points = len(y_train)+len(y_validation)+len(y_test)
    train_proportion = len(y_train)/total_points
    validation_proportion = len(y_validation)/total_points
    test_proportion = len(y_test)/total_points
    ratios =  str(round(train_proportion,2)*100)+':'+str(round(validation_proportion,2)*100)+':'+str(round(test_proportion,2)*100)
    title_text = model_type+' model with train:validation ('+ratios+') of '+str(total_points)+' datapoints'
    
    plt.text(0.5, 1.08, title_text, horizontalalignment='center',
         fontsize=14, fontweight='bold', transform = ax.transAxes)

    # Plot the data
    points_train = ax.scatter(df_train["y_train"], df_train["y_pred_train"],
                c = color_train, s = size, edgecolor = 'k', linewidths = 0.8, alpha = alpha, zorder=2)

    points_validation = ax.scatter(df_validation["y_validation"], df_validation["y_pred_validation"],
                c = color_validation, s = size, edgecolor = 'k', linewidths = 0.8, alpha = alpha, zorder=2)

    points_test = ax.scatter(df_test["y_test"], df_test["y_pred_test"],
                c = color_test, s = size, edgecolor = 'k', linewidths = 0.8, alpha = alpha, zorder=2)

    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.17),
            fancybox=True, shadow=True, ncol=5, labels=['Training','Validation','Test'])

    # Add the regression line with a confidence interval based on the training sets
    plot = sb.regplot("y_train", "y_pred_train", data=df_train, scatter=False, color=".1", 
                    truncate = True, ax=ax)

    # Title of the axis
    plot = ax.set(ylabel=f'Predicted {response_value}', xlabel=f'{response_value} from database')
    
    # Add gridlines
    ax.grid(linestyle='--', linewidth=1)

    # set limits
    size_space = 0.1*abs(min(y_train)-max(y_train))
    if min(y_train) < min(y_validation) and min(y_train) < min(y_test):
        min_value_graph = min(y_train)-size_space
    elif min(y_validation) < min(y_train) and min(y_validation) < min(y_test):
        min_value_graph = min(y_validation)-size_space
    else:
         min_value_graph = min(y_test)-size_space
        
    if max(y_train) > max(y_validation) and max(y_train) > max(y_test):
        max_value_graph = max(y_train)+size_space
    elif max(y_validation) > max(y_train) and max(y_validation) > max(y_test):
        max_value_graph = max(y_validation)+size_space
    else:
        max_value_graph = max(y_test)+size_space
        
    plt.xlim(min_value_graph, max_value_graph)
    plt.ylim(min_value_graph, max_value_graph)
        
    # save the plot a png image, type True
    plt.savefig('Predicted vs database values with test.png', dpi=400, bbox_inches='tight')

    plt.show()

    print('\nThe corresponding graph was saved in '+w_dir+'.')

elif prediction_type == 'clas':
    predictor_model = predictor_model_fun(model_type, parameters_df, random_init, prediction_type)

    predictor_model.fit(X_train_scaled, y_train)

    plot_confusion_matrix(predictor_model, X_test_scaled, y_validation,cmap='Blues') 
    plt.show()

In [ ]:
# save the predictions in a CSV file
test_data = pd.DataFrame()

test_data = X_test.copy()
test_data[f'Predicted {response_value}'] = y_pred_test
test_data['Set'] = 'Test (external)'
test_data['Name'] = test_set['Name']
test_data[response_value] = y_test

combined_data = pd.concat([df_model, test_data])


os.remove(f'{csv_training}.csv')
export_param_excel = combined_data.to_csv(f'{csv_training}_with_test.csv', index = None, header=True)
